In [ ]:
using Revise
using FUSE
using Plots;
FUSE.logging(Logging.Info; actors=Logging.Info);


In [ ]:
# sty is the act equivalent for a study, it has common parameters like server and n_workers but also study dependent parameters like n_simulations
sty,act = FUSE.study_parameters(:DatabaseGenerator);
sty

In [ ]:
# Interacting with sty
sty.server = "localhost" # this can be set to saga/omega/ your cluster
sty.n_workers = 2

sty.save_folder = <Your save folder>

sty.release_workers_after_run = true # this is the default behavior and releases workers after running the study

sty.n_simulations = 100

sty

In [ ]:
# setup ini
ini, act = FUSE.case_parameters(:ITER;init_from=:scalars)

# core profiles
ini.core_profiles.zeff = 2.5 ↔ [1.4,4.5]
ini.core_profiles.ne_setting = :greenwald_fraction
ini.core_profiles.ne_value = 0.2 ↔[0.2,1.0]

# H&CD
ini.nb_unit[1].power_launched = 33.4e6 ↔ [0.0,33.4e6]
ini.nb_unit[1].beam_energy = 1e6 ↔ [1e5,1e6]
ini.ec_launcher[1].power_launched = 20e6 ↔ [0.0,20e6]
ini.ec_launcher[1].rho_0 = 0.3 ↔ [0.1,0.9]
ini.ic_antenna[1].power_launched = 24e6 ↔ [0.0,24e6]
ini.lh_antenna[1].power_launched = 10e6 ↔ [0.0,10e6]

act.ActorPedestal.density_match =:ne_line
act.ActorCoreTransport.model = :FluxMatcher
act.ActorFluxMatcher.evolve_pedestal = false
act.ActorTGLF.warn_nn_train_bounds = false
act.ActorFluxMatcher.evolve_rotation = :fixed
act.ActorFluxMatcher.evolve_densities = :flux_match

In [ ]:
# study is the actor equivalent of actors, here sty, act and outputs are kept of the workflow
study = FUSE.StudyDatabaseGenerator(sty,ini, act); # it is possible to pass in keyword arguments to sty

# instantiating the study will also setup the study i.e. FUSE.setup(study);

In [ ]:
# for now you will have to import FUSE everywhere in your distributed computing
using Distributed
@everywhere import FUSE

# Define your workflow here, make sure it takes dd, ini , act as parameters
@everywhere function workflow_DatabaseGenerator(dd::FUSE.IMAS.dd, ini::FUSE.ParametersAllInits, act::FUSE.ParametersAllActors)
    # initialize
    FUSE.init(dd, ini, act)

    # Actors to run on the input dd
    FUSE.ActorStationaryPlasma(dd,act)

    # whatever other actors you want to run can go here

    return nothing
end

study.workflow = workflow_DatabaseGenerator

In [ ]:
FUSE.run(study);   # runs the study, this also extracts data from the simulations to a dataframe

In [ ]:
# The dataframe is stored in study, here is a summary of the dataframe
study.dataframe

In [ ]:
FUSE.analyze(study; re_extract=false)
# You can re_extract the data if you pass re_extract = false i.e. FUSE.analyze(study; re_extract=true)

In [ ]:
# All your outputs will be stored in your save location to run extract in parallel here you can do
readdir(sty.save_folder)